### Agents
Agents in essence are LLMs that accomplish a specific task. They can be supplemented with tools, structured output, and handoff to other agents.

In [2]:
%pip install openai-agents python-dotenv -qU

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from dotenv import load_dotenv
import os

# loads the .env file (if you have a global environment variable, you can skip this)
load_dotenv()

api_key = os.environ.get("OPENAI_API_KEY")

if not api_key:
    raise ValueError("OPENAI_API_KEY is not set in the environment variables")

In [4]:
from agents import Agent, Runner

agent = Agent(
    name="Basic Agent",
    instructions="You are a helpful assistant. Respond on in all caps.",
    model="gpt-4o-mini"
)

result = await Runner.run(agent, "Hello! How are you?")
result.final_output

'HELLO! I’M DOING GREAT, THANK YOU! HOW ABOUT YOU?'

In [6]:
joke_agent = Agent(
    name="Joke Agent",
    instructions="You are a joke teller. You are given a topic and you need to tell a joke about it.",
)

topic = "Boogers"
result = await Runner.run(joke_agent, topic)
result.final_output

'Why was the booger always invited to parties?\n\nBecause he was a real "pick-me-up!"'

In [7]:
language_agent = Agent(
    name="Language Agent",
    instructions="You are a language expert. You are given a joke and you need to rewrite it in a different language.",
)

joke_result = await Runner.run(joke_agent, topic)
translated_result = await Runner.run(language_agent, f"Translate this joke to Spanish: {joke_result.final_output}")
print(f"Original joke:\n{joke_result.final_output}\n")
print(f"Translated joke:\n{translated_result.final_output}")

Original joke:
Why did the booger stay in school?

Because it couldn't stop picking up new things!

Translated joke:
¿Por qué el moco se quedó en la escuela?

¡Porque no podía dejar de aprender cosas nuevas!


### Structured Outputs
Structured outputs are a way to format the output of an LLM in a structured manner. This can be useful for tasks that require specific formatting or data extraction.

In [ ]:
from pydantic import BaseModel
from agents import Agent

class Recipe(BaseModel):
    title: str
    ingredients: list[str]
    cooking_time: int # in minutes
    servings: int

recipe_agent = Agent(
    name="Recipe Agent", 
    instructions=("You are an agent for creating recipes. You will be given the name of a food and your job"
                  " is to output that as an actual detailed recipe. The cooking time should be in minutes."),
    output_type=Recipe
)

response = await Runner.run(recipe_agent, "Italian Sasuage with Spaghetti")
recipe = response.final_output

Recipe(title='Italian Sausage with Spaghetti', ingredients=['400g spaghetti', '4 Italian sausages, casings removed', '2 tablespoons olive oil', '1 onion, finely chopped', '2 cloves garlic, minced', '1 teaspoon chili flakes (optional)', '1 can (400g) diced tomatoes', '1 tablespoon tomato paste', '1 teaspoon dried oregano', 'Salt and pepper to taste', 'Parmesan cheese, grated (for serving)', 'Fresh basil leaves (for garnish)'], cooking_time=30, servings=4)

### Tool Calling
Tool calling is a way to extend the capabilities of an LLM by allowing it to call external tools or APIs. This can be useful for tasks that require access to external data or services.

In [20]:
from agents import Agent, function_tool

@function_tool
def get_weather(city: str) -> str:
    print(f"Getting weather for {city}")
    return "sunny"

@function_tool
def get_temperature(city: str) -> str:
    print(f"Getting temperature for {city}")
    return "70 degrees"

agent = Agent(
    name="Weather Agent",
    instructions="You are the local weather agent. You are given a city and you need to tell the weather and temperature. For any unrelated queries, say I cant help with that.",
    tools=[get_weather, get_temperature]
)

result = await Runner.run(agent, "Dallas")
result.final_output

Getting weather for Dallas
Getting temperature for Dallas


'The weather in Dallas is currently sunny, with a temperature of 70 degrees.'

In [21]:
from agents import WebSearchTool

news_agent = Agent(
    name="News Reporter",
    instructions="You are a news reporter. Your job is to find recent news articles on the internet about US politics.",
    tools=[WebSearchTool()]
)

result = await Runner.run(news_agent, "find news")
print(result.final_output)

Here are some recent developments in U.S. politics:

**Democratic Party's Internal Challenges**

The Democratic Party is facing internal divisions, particularly highlighted during recent budget battles. Some members supported Republican bills, while others opposed them, complicating the party's unified stance against the Trump administration. Calls for new leadership, including critiques of figures like Chuck Schumer, have emerged. The party is urged to establish a clear message and engage voters actively ahead of the midterm elections. ([ft.com](https://www.ft.com/content/13b4c22d-23e0-4f62-bd5f-5a7afd60e83d?utm_source=openai))

**Donald Trump's Influence on Cultural Institutions**

Former President Donald Trump continues to assert influence over American institutions. As chair of the John F. Kennedy Center for the Performing Arts, he aims to remove "woke" influences and discourage performances he deems "anti-American." This cultural campaign parallels actions in academia, particularl